<a href="https://colab.research.google.com/github/ngzhankang/Deep-Learning/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# to ensure that the current gpu utilization is 0

# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

Gen RAM Free: 12.8 GB  | Proc size: 113.3 MB
GPU RAM Free: 15079MB | Used: 0MB | Util   0% | Total 15079MB


In [ ]:
# if utilization is > 0, run this code(keep running this cell and the above cell till the util number is 0%):
# NOTE THAT RUNNING THIS MIGHT KILL GPU SESSION AND RESULT IN DATA LOSS
!kill -9 -1

# Assignment 1
Submitted by : P1935727 Ng Zhan Kang<br>
Class of DIT/FT/2B/11


# Index
1. [About The RMS Titanic (Background Information)](#About-The-RMS-Titanic-(Background-Information))
2. [Data Importing](#Data-Importing)
3. [Exploratory Data Analysis](#Exploratory-Data-Analysis)
4. [Data Preparation](#Data-Preparation)<br>
    4.1. [Handle missing data](#Handle-missing-data)<br>
    4.2. [Converting Features (Feature Selection)](#Converting-Features-(Feature-Selection))<br>
    4.3. [Correcting Data](#Correcting-Data)<br>
    4.3. [Drop Columns](#Drop-Columns)<br>
5. [Model Training](#Model-Training)<br>
    5.1. [Split Data into Train and Test Sets](#Split-Data-into-Train-and-Test-Sets)<br>
    5.2. [Train Model with Algorithm: Logistics Regression](#Train-Model-with-Algorithm:-Logistics-Regression)<br>
    5.3. [Train Model with Algorithm: Random Forest Classifier](#Train-Model-with-Algorithm:-Random-Forest-Classifier)<br>
    5.4. [Train Model with Algorithm: Nu-support vector Classification (NSVC)](#Train-Model-with-Algorithm:-Nu-support-vector-Classification-(NSVC))<br>
    5.5. [Train Model with Algorithm: Gaussian Naive Bayes](#Train-Model-with-Algorithm:-Gaussian-Naive-Bayes)<br>
6. [Score and Evaluate Model](#Score-and-Evaluate-Model)<br>
    6.1. [Score Model and Evaluate Model with: Logistics Regression](#Score-Model-and-Evaluate-Model-with:-Logistics-Regression)<br>
    6.2. [Score Model and Evaluate Model with: Random Forest Classifier](#Score-Model-and-Evaluate-Model-with:-Random-Forest-Classifier)<br>
    6.3. [Score Model and Evaluate Model with: Nu-support vector Classification (NSVC)](#Score-Model-and-Evaluate-Model-with:-Nu-support-vector-Classification-(NSVC))<br>
    6.4. [Score Model and Evaluate Model with: Gaussian Naive Bayes](#Score-Model-and-Evaluate-Model-with:-Gaussian-Naive-Bayes)<br>
7. [Submitting to Kaggle](#Submitting-to-Kaggle)<br>
    7.1. [Stupid Baseline (Everyone Dies)](#Stupid-Baseline-(Everyone-Dies))<br>
    7.2. [Data Preparation for the test.csv](#Data-Preparation-for-the-test.csv)<br>
    7.3. [Prediction with Logistics Regression Trained Model](#Prediction-with-Logistics-Regression-Trained-Model)<br>
    7.4. [Prediction with Random Forest Classifier Trained Model](#Prediction-with-Random-Forest-Classifier-Trained-Model)<br>
    7.5. [Prediction with Nu-support vector Classification (NSVC) Trained Model](#Prediction-with-Nu-support-vector-Classification-(NSVC)-Trained-Model)<br>
    7.6. [Prediction with Gaussian Naive Bayes Trained Model](#Prediction-with-Gaussian-Naive-Bayes-Trained-Model)<br>
8. [Conclusions](#Conclusions)
9. [References](#References)

# About The RMS Titanic (Background Information)
- BACKGROUND
    - Owned by White Star Line, a British shipping company which was one of the most prominent shipping lines in the world providing passenger and cargo shipping between the British Empire and the United States.
    - Was one of the 3 Olympic-class ocean liners(namely RMS Olympic, RMS Titanic and HMHS Britannic).
    - It's original name was in fact "Number 401" and design was based on the revised design of "Olympic".
    - Was built by the Harland and Wolff shipyard in Belfast, Northern Ireland.
    - Was commanded by Captain Edward Smith, who would eventually sink down with the ship.
    - Was the largest ship afloat at the time she entered service, which was 1911.
    - Launched on 31 May 1911, laid down on 31 March 1909.
    - Port of registry is at Liverpool, United Kingdom.
    - Cost to build was GB£1.5 million, or SGD 266 million in SGD in 2020.
    - Maiden voyage was on 10 April 1912.
    - Sunk on 14 April 1912 23:40PM (Ship's time) and sank 2h 40 min later on 15 April 1912, 5 days from her maiden voyage due to a collison against an iceberg.


- FEATURES
    - Powered by steam engines and 4-cylinders, with a total power output of 34000kW.
    - Heated by burning coal.
    - Included a steam-driven electical plant, capable of producing more power than an average city power station of the time, and was used to provide electricity for the ship.
    - Was notably the 'unsinkable' due to the interior of the ship subdivided into 16 primary compartments divided into 15 bulkheads extended above the waterline, which 11 vertically closing waterthight doors could seal off the compartments in the event of an   emergency.
    - Had her own waterworks capable of heating and pumping water to all parts of the vessel via complex pipes and valves. The first class cabins were fitted with additional electical heaters.
    - Able to accomodate 833 1st class passengers, 614 2nd class passengers and 1006 3rd class passengers. In total a capacity of 2453 passengers. In fact, her max capacity is 3547 according to most of her original configuration documents.
    - Have a total of 20 lifeboats(14 wooden Harland & Wolff lifeboats, capacity of 65ppl),(4 Engelhardt boats, capacity of 47ppl), 2 emergency cutters(like a middle sized sailcraft) that can hold 40 ppl.
    - Have the capacity to carry 64 lifeboats in fact, enough to carry 4000 passengers, more than her actual capacity.
    - At that time, lifeboats were **intended to ferry survivors from a sinking ship to a rescuing ship, not keep afloat the whole population or power them to the shore.**
    
    
- MAIDEN VOYAGE
    - Liverpool was registered as her home port.
    - Was intended to be the 1st of the many trans-Atlantic crossing between Southampton and New York via Cherbough and Queesntownon westbound runs, returning via Plymouth in England while eastbound.
    - Had around 885 crew members on board, which are not permanent crew, which many were casual workers who only came abroad the ship a few hours before she sailed to  Southampton.
    - Captain Edward Smith, who was transferred from the *Olympic* to take command for *Titanic*, was the ship commander.
    - 97% of the crew were male, with 23 of them just females.
    - Total passengers are around 1,317 people: 324 in First Class, 284 in Second Class, and 709 in Third Class. Of these, 869 (66%) were male and 447 (34%) female. There were 107 children aboard, the largest number of whom were in Third Class. The ship was considerably under capacity on her maiden voyage, as she could accommodate 2,453 passengers—833 First Class, 614 Second Class, and 1,006 Third Class.
    - Notable passengers are American millionaire John Jacob Astor IV, who unfortunately did not survive and his wife Madeleine Force Astor, who survived the disaster.
    - There were 3 ports that Titanic collected its passenger. The 1st port was Southampton Terminus railway station, alongside Titanic's berth, where the majority 3rd class passengers were to board, and the 1st and 2nd class passengers also boarded.
    - The 2nd port was Cherbourg and Queenstown where additonal passengers were picked up.
    - Titanic started her maiden voyage at noon, as scheduled.
    
    
- ATLANTIC CROSSING
    - Titanic was planned to arrive at New York Pier 59, or modern Chelsea Piers in NYC on the morning of 17 April. She travelled roughly 2643 nautical miles in total from Queenstown to Nantucket Shoals Light where she made her fatal contact with an iceberg.
    - If Titanic were to not meet with an accident, it would only take a mere 193 nautical miles more to Ambrose Light and finally New York Harbour.
    - There had been series or warning made by ships in the area of the Grand Banks of Newfoundland, near to where drifting ice were spotted. But the Titanic decided to still steam at full speed, which was a common practice back then. Ships were also operated close to full speed. Ice was posed to be little danger to large vessels, or at least not disastrous.
    
    
- SINKING
    - At 23:40 on April 14, Frederick Fleet, the lookout spotted a iceberg immediately ahead of Titanic and alerted the bridge. First Officer William Murdoch ordered the ship to be steered around the obstacle and the engines to be stopped, but it was too late for the starboard side of Titanic struck the iceberg, creating a series of holes below the waterline, causing water to seep in. Soon , 5 of the ship's watertight compartments were breached. It soon became clear that the ship was doomed, as she could not survive more than four compartments being flooded. Titanic began sinking bow-first, with water spilling from compartment to compartment as her angle in the water became steeper.
    - Crew members were not prepared to handle such accidents. Titanic only had enough lifeboats to carry about half of those on board. Some lifeboats were launched only barely half-full, even if they could have rescued more passengers. Third-class passengers were largely left to fend for themselves, causing many of them to become trapped below decks as the ship filled with water.
    - The "women and children first" protocol was generally followed when loading the lifeboats, and most of the male passengers and crew were left aboard.
    - Distress signals were sent by wireless, rockets, and lamp, but none of the ships that responded was near enough to reach Titanic before she sank. The SS Californian, which was the last to have been in contact before the collision, saw Titanic's flares but failed to assist.
    - A radio operator on board the Birma, for instance, estimated that it would be 6 a.m. before the liner could arrive at the scene.
  
  
- SURVIVORS
    - About 710 people survived the disaster and were conveyed by Carpathia to New York, Titanic's original destination, while at least 1,500 people lost their lives.
    - Fewer than a third of those aboard Titanic survived the disaster. Some survivors died shortly afterwards; injuries and the effects of exposure caused the deaths of several of those brought aboard Carpathia.
    - The last living survivor, Millvina Dean from England, who at only nine weeks old was the youngest passenger on board, died aged 97 on 31 May 2009.
    
    
- GRAPHICAL IMAGE
    <img src="https://images.squarespace-cdn.com/content/5006453fe4b09ef2252ba068/1351660113175-514SN9PXFWB9N2MNB8DV/TItanic-Survival-Infographic.jpg?format=1500w&content-type=image%2Fjpeg" alt="Graphical Image Of How Titanic Sunk" title="Title text" />


## Titanic
- For this problem, use only 1 target output variable (Survived)
- Prepare the data and perform necessary feature engineering
- Create **_at least two models_** 
- Evaluate the models and pick the best candidate
- Form some conclusions

# Data Importing

In [4]:
# Suppress Future Warnings
import warnings
warnings.filterwarnings('ignore')

In [6]:
# check versions of libraries we are going to use
import torch
import numpy as np
import pandas as pd
import matplotlib
import platform

message="        Versions        "
print("*"*len(message))
print(message)
print("*"*len(message))
print("PyTorch version={}".format(torch.__version__))
print("Numpy version={}".format(np.__version__))
print("Pandas version={}".format(pd.__version__))
print("Matplotlib version={}".format(matplotlib.__version__))
print("Python version={}".format(platform.python_version()))

************************
        Versions        
************************
PyTorch version=1.6.0+cu101
Numpy version=1.18.5
Pandas version=1.1.2
Matplotlib version=3.2.2
Python version=3.6.9


In [7]:
# start importing necessary libraries
import torch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# get the dataset from pytorch library
